In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

from sklearn.model_selection import KFold

/kaggle/input/linking-writing-processes-to-writing-quality/sample_submission.csv
/kaggle/input/linking-writing-processes-to-writing-quality/test_logs.csv
/kaggle/input/linking-writing-processes-to-writing-quality/train_scores.csv
/kaggle/input/linking-writing-processes-to-writing-quality/train_logs.csv


## データの読み込み

In [3]:
# 訓練データ読み込み
train_df = pd.read_csv("/kaggle/input/linking-writing-processes-to-writing-quality/train_logs.csv")
train_scores = pd.read_csv("/kaggle/input/linking-writing-processes-to-writing-quality/train_scores.csv")
# 検証データ読み込み
test_df = pd.read_csv("/kaggle/input/linking-writing-processes-to-writing-quality/test_logs.csv")

print("train_df： ", train_df.shape)
print("train_scores: ", train_scores.shape)
print("test_df: ", test_df.shape)

train_df：  (8405898, 11)
train_scores:  (2471, 2)
test_df:  (6, 11)


In [4]:
# 訓練データの表示確認
train_df.head()

,id,event_id,down_time,up_time,action_time,activity,down_event,up_event,text_change,cursor_position,word_count
0,001519c8,1,4526,4557,31,Nonproduction,Leftclick,Leftclick,NoChange,0,0
1,001519c8,2,4558,4962,404,Nonproduction,Leftclick,Leftclick,NoChange,0,0
2,001519c8,3,106571,106571,0,Nonproduction,Shift,Shift,NoChange,0,0
3,001519c8,4,106686,106777,91,Input,q,q,q,1,1
4,001519c8,5,107196,107323,127,Input,q,q,q,2,1


In [5]:
# 検証データの表示確認
test_df.head()

,id,event_id,down_time,up_time,action_time,activity,down_event,up_event,text_change,cursor_position,word_count
0,0000aaaa,1,338433,338518,85,Input,Space,Space,,0,0
1,0000aaaa,2,760073,760160,87,Input,Space,Space,,1,0
2,2222bbbb,1,711956,712023,67,Input,q,q,q,0,1
3,2222bbbb,2,290502,290548,46,Input,q,q,q,1,1
4,4444cccc,1,635547,635641,94,Input,Space,Space,,0,0


## データ概要 

In [6]:
# 各項目のデータ型の確認
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8405898 entries, 0 to 8405897
Data columns (total 11 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   id               object
 1   event_id         int64 
 2   down_time        int64 
 3   up_time          int64 
 4   action_time      int64 
 5   activity         object
 6   down_event       object
 7   up_event         object
 8   text_change      object
 9   cursor_position  int64 
 10  word_count       int64 
dtypes: int64(6), object(5)
memory usage: 705.5+ MB


In [7]:
# データの統計データを確認
train_df.describe()

,event_id,down_time,up_time,action_time,cursor_position,word_count
count,8.405898e+06,8.405898e+06,8.405898e+06,8.405898e+06,8.405898e+06,8.405898e+06
mean,2.067649e+03,7.935603e+05,7.936584e+05,9.808498e+01,1.222964e+03,2.314687e+02
std,1.588284e+03,5.149451e+05,5.149428e+05,2.533985e+02,9.485242e+02,1.759088e+02
min,1.000000e+00,1.060000e+02,2.520000e+02,0.000000e+00,0.000000e+00,0.000000e+00
25%,8.520000e+02,3.731842e+05,3.732820e+05,6.600000e+01,4.990000e+02,9.600000e+01
50%,1.726000e+03,7.208860e+05,7.209800e+05,9.300000e+01,1.043000e+03,2.000000e+02
75%,2.926000e+03,1.163042e+06,1.163141e+06,1.220000e+02,1.706000e+03,3.270000e+02
max,1.287600e+04,8.313630e+06,8.313707e+06,4.474700e+05,7.802000e+03,1.326000e+03


In [8]:
# 欠損値の確認(訓練データ)
for col in train_df.columns:
    pct_missing = np.mean(train_df[col].isnull())
    print('{} - {}%'.format(col, round(pct_missing*100)))

id - 0%
event_id - 0%
down_time - 0%
up_time - 0%
action_time - 0%
activity - 0%
down_event - 0%
up_event - 0%
text_change - 0%
cursor_position - 0%
word_count - 0%


In [9]:
# 欠損値の確認(検証データ)
for col in test_df.columns:
    pct_missing = np.mean(test_df[col].isnull())
    print('{} - {}%'.format(col, round(pct_missing*100)))

id - 0%
event_id - 0%
down_time - 0%
up_time - 0%
action_time - 0%
activity - 0%
down_event - 0%
up_event - 0%
text_change - 0%
cursor_position - 0%
word_count - 0%


## 特徴量の作成
https://www.kaggle.com/code/polyakovstepan/random-forest-atboost-xgb-baseline

In [10]:
def summary_time(df):
    result = df.groupby('id')['action_time'].sum().reset_index()
    result.rename(columns={'action_time': 'summary_time'}, inplace=True)
    return result

def start_pause(df):
    result = df.groupby('id')['down_time'].min().reset_index()
    result.rename(columns={'down_time':'start_pause'}, inplace=True)
    return result

def enter_click(df):
    copy_df = df
    copy_df['enter_click'] = (copy_df['down_event'] == 'Enter')
    copy_df = copy_df.groupby('id')['enter_click'].sum().reset_index()
    return copy_df

def space_click(df):
    copy_df = df
    copy_df['space_click'] = (copy_df['down_event'] == 'Space')
    copy_df = copy_df.groupby('id')['space_click'].sum().reset_index()
    return copy_df

def backspace_click(df):
    copy_df = df
    copy_df['backspace_click'] = (copy_df['down_event'] == 'Backspace')
    copy_df = copy_df.groupby('id')['backspace_click'].sum().reset_index()
    return copy_df

def symbol_length(df):
    result = df.groupby('id')['cursor_position'].max().reset_index()
    result.rename(columns={'cursor_position':'symbol_length'}, inplace=True)
    return result

def text_length(df):
    result = df.groupby('id')['word_count'].max().reset_index()
    return result

def nonproduction_feature(df):
    result = df.groupby('id')['activity'].apply(lambda x: (x == 'Nonproduction').mean() * 100).reset_index()
    result.rename(columns={'activity': 'nonproduction_feature'}, inplace=True)
    return result

def input_feature(df):
    result = df.groupby('id')['activity'].apply(lambda x: (x == 'Input').mean() * 100).reset_index()
    result.rename(columns={'activity': 'input_feature'}, inplace=True)
    return result

def remove_feature(df):
    result = df.groupby('id')['activity'].apply(lambda x: (x == 'Remove/Cut').mean() * 100).reset_index()
    result.rename(columns={'activity': 'remove_feature'}, inplace=True)
    return result

def mean_action_time(df):
    result = df.groupby('id')['action_time'].mean().reset_index()
    result.rename(columns={'action_time':'mean_action_time'}, inplace=True)
    return result

def replace_feature(df):
    result = df[df['activity'] == 'Replace'].groupby('id').size().reset_index(name='replace_feature')
    return result

def text_change_unique(df):
    result = df.groupby('id')['text_change'].nunique().reset_index()
    result.rename(columns={'text_change': 'tch_unique'}, inplace=True)
    return result

def sentence_size_feature(df):
    result = df[(df['text_change'] == '.') & (df['down_event'] != 'Backspace')].groupby('id').size().reset_index(name = 'number_sentence')
    return result

In [14]:
def getDataset(train_df):
    new_df = summary_time(train_df)
    
    functions = [
        start_pause,enter_click,space_click,backspace_click,symbol_length,text_length,
        nonproduction_feature, input_feature, remove_feature, mean_action_time, replace_feature,
        text_change_unique, sentence_size_feature
    ]
    
    for func in functions:
        result_df = func(train_df)
        new_df = pd.merge(new_df, result_df, on='id', how='outer')
        
    return new_df

In [15]:
# 訓練データの特徴量生成
df = getDataset(train_df)
print('train_info: ', df.shape)
df.head()

train_info:  (2471, 15)


,id,summary_time,start_pause,enter_click,space_click,backspace_click,symbol_length,word_count,nonproduction_feature,input_feature,remove_feature,mean_action_time,replace_feature,tch_unique,number_sentence
0,001519c8,297243,4526,4,357,417,1539,256,4.693000,78.607743,16.308174,116.246774,7.0,17,21.0
1,0022f953,275391,30623,6,391,260,1676,323,10.350448,78.973105,10.594947,112.221271,1.0,12,15.0
2,0042269b,421201,4441,17,552,439,2291,404,4.231141,84.985493,10.614120,101.837766,7.0,19,21.0
3,0059420b,189596,41395,3,243,152,1047,206,6.362468,83.804627,9.704370,121.848329,1.0,10,13.0
4,0075873a,313702,78470,10,324,517,1402,252,2.844725,76.728566,20.426709,123.943896,NaN,9,23.0


In [16]:
# 検証データの特徴量生成
test = getDataset(test_df)
print('test_info: ', test.shape)
test.shape

test_info:  (3, 15)


(3, 15)

## 欠損値消去

In [17]:
# 訓練データの欠損値確認
for col in df.columns:
    pct_missing = np.mean(df[col].isnull())
    print('{} - {}%'.format(col, round(pct_missing*100)))

id - 0%
summary_time - 0%
start_pause - 0%
enter_click - 0%
space_click - 0%
backspace_click - 0%
symbol_length - 0%
word_count - 0%
nonproduction_feature - 0%
input_feature - 0%
remove_feature - 0%
mean_action_time - 0%
replace_feature - 56%
tch_unique - 0%
number_sentence - 0%


In [18]:
# 検証データの欠損値確認
for col in test.columns:
    pct_missing = np.mean(test[col].isnull())
    print('{} - {}%'.format(col, round(pct_missing*100)))

id - 0%
summary_time - 0%
start_pause - 0%
enter_click - 0%
space_click - 0%
backspace_click - 0%
symbol_length - 0%
word_count - 0%
nonproduction_feature - 0%
input_feature - 0%
remove_feature - 0%
mean_action_time - 0%
replace_feature - 100%
tch_unique - 0%
number_sentence - 100%


In [19]:
# replace_featureとnumber_sentenceに欠損値が確認される
# この２つの特徴量を消去する
df_numeric = df.select_dtypes(include=[np.number])
numeric_cols = df_numeric.columns.values

for col in  numeric_cols:
    missing = df[col].isnull()
    num_missing = np.sum(missing)
    
    if num_missing > 0:
        print('imputing missing values for: {}'.format(col))
        df['{}_ismissing'.format(col)] = missing
        med = test[col].median()
        df[col] = df[col].fillna(med)

imputing missing values for: replace_feature
imputing missing values for: number_sentence


/opt/conda/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/opt/conda/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
